In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../src/')  
from scraper.session import login, BASE_URL
from scraper.fetch import get_court_links_and_dates
# from scraper.parser import parse_cases
from scraper.court_scraper import CourtScraper

from db.db_methods import get_connection, get_court_id_by_city, insert_court_case
from db.models import CourtCase



# getting court links doesnt need session as no log in... but it does want you to log in to view the deetts..
links_and_dates = get_court_links_and_dates()


getting court links


In [3]:

# logging in
session = login()      





Status Code: 302
Final URL: https://courtserve.net/
session cookies: <RequestsCookieJar[<Cookie SITELOKPW=cjV6cEpPQldGNDlRK3BHcWhiYXFNdjZscUZJQ0cwRytTcG5TdkVvPQ%3D%3D for .courtserve.net/>, <Cookie PHPSESSID=1f3a81bd64be6cb82db08b624802d5dda5d70d66 for courtserve.net/>]>
probs logged in


In [4]:
# get db connection
get_connection()


<connection object at 0x77f04c59efc0; dsn: 'user=jonah password=xxx dbname=courtdb_dev host=localhost port=65432', closed: 0>

In [ ]:

for i, (link, date) in enumerate (links_and_dates):
    if i < 2:
        continue
    courtScraper = CourtScraper(session, BASE_URL + link)
    if not courtScraper.load_case_page():
        print(f"struggle to find 'new tab url' for {link}")
        continue
    courtScraper.get_case_list_soup()
    courtScraper.extract_city_and_court_name()
    court_cases = courtScraper.rows_to_objects(date)
    if not court_cases:
        continue
    for case in court_cases: # iterate through scraped court cases and add to db
        court_id = get_court_id_by_city(case.city)
        if case.city == 'Barrow':
            print("yippee")
        if not court_id and case.city:
            print(f"no court id for {case.city}")
            continue 
        insert_court_case(case, court_id)


yippee
yippee
yippee
yippee
yippee
case already exists?: CourtCase(case_id='BY25F00004', start_time_string='12:00 PM', date='17/04/25', duration='30 minutes', claimant='H Hirst', defendant='T Wilkinson- SPM', hearing_type='Injunction (Further Hearing)', hearing_channel='In Person, Video - CVP', city='Barnsley', start_time_epoch=1744891200)
 <built-in method with_traceback of UniqueViolation object at 0x77f044d027c0>
case already exists?: CourtCase(case_id='L8QZ3Q90', start_time_string='11:00 AM', date='16/04/25', duration='1\r\n    hour 30 minutes', claimant='Luke Har', defendant='ey v Watford Insurance Co Europe ltd', hearing_type='Small Claims', hearing_channel='In Person', city='', start_time_epoch=1744801200)
 <built-in method with_traceback of NotNullViolation object at 0x77f044204640>
case already exists?: CourtCase(case_id='M00BP036', start_time_string='12:30 PM', date='16/04/25', duration='10     minutes', claimant='Jasmin Coaker', defendant='Planet Moto ltd', hearing_type='Thi